In [ ]:
!pip install pystac_client rasterio -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 12.8 MB/s eta 0:00:00


In [ ]:
# Libs
import os
import requests
import pystac_client
import rasterio
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio.io import MemoryFile
from rasterio.mask import mask
from rasterio.windows import from_bounds
from pyproj import Transformer, CRS
from tqdm import tqdm
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling

In [ ]:
def get_evi_cube_from_bdc(geojson_file, collection='S2-16D-2', datetime_range="2024-05-01/2024-09-30", output_multiband="evi_cube.tif"):
    # Conectar ao STAC do Brazil Data Cube
    service = pystac_client.Client.open('https://data.inpe.br/bdc/stac/v1/')

    # Carregar geojson
    gdf = gpd.read_file(geojson_file)
    bbox = gdf.total_bounds

    # Buscar imagens que intersectam a área
    item_search = service.search(
        bbox=bbox,
        datetime=datetime_range,
        collections=[collection]
    )

    items = list(item_search.items())
    if not items:
        print("Nenhuma imagem encontrada.")
        return

    print(f"{len(items)} imagens encontradas.")

    evi_bands = []
    transform, crs = None, None

    for i, item in enumerate(items):
        assets = item.assets

        if 'EVI' not in assets:
            print(f"Aviso: Item {item.id} não possui EVI.")
            continue

        asset_url = assets['EVI'].href  # URL direta da imagem

        with rasterio.open(asset_url) as src:
            try:
                # Verificar e reprojetar para WGS84, se necessário
                if not src.crs or src.crs.to_epsg() != 4326:
                    dst_crs = CRS.from_epsg(4326)
                    transform, width, height = calculate_default_transform(
                        src.crs, dst_crs, src.width, src.height, *src.bounds
                    )

                    kwargs = src.meta.copy()
                    kwargs.update({
                        'crs': dst_crs,
                        'transform': transform,
                        'width': width,
                        'height': height
                    })

                    # Criar um arquivo temporário na memória para armazenar a reprojeção
                    memfile = MemoryFile()
                    with memfile.open(**kwargs) as dst:
                        for j in range(1, src.count + 1):
                            reproject(
                                source=rasterio.band(src, j),
                                destination=rasterio.band(dst, j),
                                src_transform=src.transform,
                                src_crs=src.crs,
                                dst_transform=transform,
                                dst_crs=dst_crs,
                                resampling=Resampling.nearest
                            )

                    # Abrir novamente o dataset para evitar o fechamento prematuro
                    src_reprojected = memfile.open()

                else:
                    src_reprojected = src  # Se já estiver em WGS84, usar direto

                # Agora podemos cortar a imagem sem erro
                evi, transform = mask(src_reprojected, gdf.geometry, crop=True)
                evi = evi[0] * 0.0001  # Ajuste da escala

                # Fechar corretamente a memória ao final do uso
                if 'memfile' in locals():
                    memfile.close()

                # Substituir valores inválidos por NaN
                evi = np.where(evi != -9999, evi, np.nan)

                evi_bands.append(evi)
                crs = src.crs  # Salvar CRS para o arquivo final

            except ValueError as e:
                print(f"Erro ao recortar imagem {i}: {e}")

    # Se nenhuma imagem foi processada, encerrar
    if not evi_bands:
        print("Nenhuma imagem válida para salvar.")
        return

    # Converter lista para array numpy com shape adequado (bandas, altura, largura)
    evi_array = np.stack(evi_bands, axis=0)

    # Salvar o cubo EVI como um único arquivo multibanda
    save_multiband_image(evi_array, transform, crs, output_multiband)
    print(f"Cubo EVI salvo como: {output_multiband}")


def save_multiband_image(data, transform, crs, output_path):
    """ Salva um array numpy como um único GeoTIFF multibanda """
    with rasterio.open(
        output_path, 'w',
        driver='GTiff',
        height=data.shape[1],  # Altura
        width=data.shape[2],   # Largura
        count=data.shape[0],   # Número de bandas
        dtype=data.dtype,
        crs=crs,
        transform=transform
    ) as dst:
        for i in range(data.shape[0]):
            dst.write(data[i], i + 1)

    print(f"Arquivo multibanda salvo: {output_path}")

In [ ]:
# Caminho do GeoJSON
geojson_file = "/content/PNB.geojson"

# Obter stack EVI
evi_stack = get_evi_cube_from_bdc(geojson_file)

11 imagens encontradas.
Arquivo multibanda salvo: evi_cube.tif
Cubo EVI salvo como: evi_cube.tif


In [ ]:
def plot_multiband_tiff(tiff_path, nodata_value=-0.9999):
    """Lê um arquivo GeoTIFF multibanda e plota as bandas em 2 linhas e 5 colunas."""

    with rasterio.open(tiff_path) as src:
        num_bands = src.count
        fig, axes = plt.subplots(2, 5, figsize=(20, 15))

        axes = axes.flatten()

        for i in range(num_bands):
            band = src.read(i + 1)

            # Máscara para valores nodata
            band = np.where(band == nodata_value, np.nan, band)

            im = axes[i].imshow(band, cmap='RdYlGn', interpolation='nearest')
            axes[i].set_title(f'Banda {i+1}')
            axes[i].axis('off')
            fig.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

        # Oculta eixos extras
        for j in range(num_bands, len(axes)):
            fig.delaxes(axes[j])

        plt.tight_layout()
        plt.show()

In [ ]:
# Exemplo de uso
plot_multiband_tiff("/content/evi_cube.tif")

In [ ]:
def plot_median_per_band(tiff_path, nodata_value=-0.9999):
    """Lê um arquivo GeoTIFF multibanda, calcula a mediana de cada banda e plota um gráfico de linha."""

    with rasterio.open(tiff_path) as src:
        num_bands = src.count
        medians = []

        for i in range(num_bands):
            band = src.read(i + 1)

            # Máscara para valores nodata
            band = band[band != nodata_value]

            # Calcula a mediana ignorando NaNs
            medians.append(np.median(band) if band.size > 0 else np.nan)

    # Plotando o gráfico
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, num_bands + 1), medians, marker='o', linestyle='-', color='g', label='Mediana (EVI)')

    # Ajustes do gráfico
    plt.xlabel("Banda")
    plt.ylabel("EVI")
    plt.title("Mediana por Banda")
    plt.xticks(range(1, num_bands + 1))
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend()
    plt.show()

In [ ]:
# Exemplo de uso
plot_median_per_band("/content/evi_cube.tif")